In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
from scipy.stats import uniform
from datetime import datetime as dt
import math
from tqdm import tqdm

rd.seed(0)

import linear

In [ ]:
dim = 500 # dimension of data 
mu_norm = 2 # mu_norm 
mu_dist = 1
theta_init_sd = 0.1

#generate mu_1
mu = np.random.standard_normal(dim) 
mu_n = np.linalg.norm(mu)
mu_1 = mu*mu_norm/mu_n 

mu_diff = np.random.standard_normal(dim)
mu_2 = mu_1 + mu_dist * mu_diff / np.linalg.norm(mu_diff)


n11 = 2000 ; n12 = 1000   # data for task 1
n21 = [10,50,100,150,200,250,300,350,400,450,500,600,700,800,900,1000,2000]
n22 = 5000   # data for task 2

In [ ]:
# initialize theta
theta = theta_init_sd * np.random.standard_normal(dim+1).reshape(dim+1,1)

alpha = 0.1

# epoch for transfer method 
epochs1 = 300
epochs2 = 300
mini_batch_size = 20

# test the error based on three method
error_1 = []
error_2 = []
error_3 = []
error_4 = []
error_5 = []

for i in range(len(n21)):
    # generate data
                    # train1: training dataset for task1
                    # train2: training dataset for task2
                    # test1: test dataset for task1
                    # test2: test dataset for task2
    descent = linear.DescentMethod(n11,n12,n21[i],n22,dim,mu_norm,mu_1,mu_2)
    train1,test1,train2,test2 = descent.loaddataset()
    
    #transfer method (lambda=[0,1])
    _,_,weights_SGD1 = \
    descent.MIXSGD(train1,train2,test1,test2,epochs1,mini_batch_size,alpha,theta,lmbda=0)
    cost_SGD1,error_SGD1,_ = \
    descent.MIXSGD(train1,train2,test1,test2,epochs2,mini_batch_size,alpha,weights_SGD1,lmbda=1)
    error_1.append(error_SGD1[-1])
    
    #mixture transfer method (lambda=[0.8,0.8])
    cost_SGD2,error_SGD2,weights_SGD2 = \
    descent.MIXSGD(train1,train2,test1,test2,epochs1+epochs2,mini_batch_size,alpha,theta,lmbda=0.8)
    
    error_2.append(error_SGD2[-1])
    
    # method 3 (lambda=[0,0.8])
    _,_,weights_SGD3 = \
    descent.MIXSGD(train1,train2,test1,test2,epochs1,mini_batch_size,alpha,theta,lmbda=0)
    cost_SGD3,error_SGD3,_ = \
    descent.MIXSGD(train1,train2,test1,test2,epochs2,mini_batch_size,alpha,weights_SGD3,lmbda=0.8)
    error_3.append(error_SGD3[-1])
    
    # method 4 (lambda=[0.8,1])
    _,_,weights_SGD4 = \
    descent.MIXSGD(train1,train2,test1,test2,epochs1,mini_batch_size,alpha,theta,lmbda=0.8)
    cost_SGD4,error_SGD4,_ = \
    descent.MIXSGD(train1,train2,test1,test2,epochs2,mini_batch_size,alpha,weights_SGD4,lmbda=1)
    error_4.append(error_SGD4[-1])
    
    # method 5 (lambda = [1,1])
    cost_SGD5,error_SGD5,weights_SGD5 = \
    descent.MIXSGD(train1,train2,test1,test2,epochs1+epochs2,mini_batch_size,alpha,theta,lmbda=1)
    
    error_5.append(error_SGD5[-1])

In [ ]:
fig, ax = plt.subplots(figsize = (12, 8))
n21 = [10,50,100,150,200,250,300,350,400,450,500,600,700,800,900,1000,2000]
x = range(len(n21))
ax.plot(x,error_1,label='lambda=[0,1]')
ax.plot(x,error_2,label='lambda=[0.8,0.8]')
# ax.plot(x,error_3,label='lambda=[0,0.8]')
# ax.plot(x,error_4,label='lambda=[0.8,1]')
# ax.plot(x,error_5,label='lambda=[1,1]')
plt.xticks(x,n21)
ax.set_xlabel('Training Dataset Size')
ax.set_ylabel('Test Error')
ax.legend()
plt.show()